# Описание датасета, EDA

Датасет содержит информацию о клиентах банка, которые либо ушли, либо остались клиентами банка

Список фичей:

    •   Customer ID: Уникальный идентификатор каждого клиента.
    •   Surname: Фамилия клиента.
    •   Credit Score: Числовое значение, представляющее кредитный рейтинг клиента.
    •   Geography: Страна проживания клиента (Франция, Испания или Германия).
    •   Gender: Пол клиента (Мужской или Женский).
    •   Age: Возраст клиента.
    •   Tenure: Количество лет, которое клиент обслуживается в банке.
    •   Balance: Баланс на счёте клиента.
    •   NumOfProducts: Количество банковских продуктов, которыми пользуется клиент (например, сберегательный счёт, кредитная карта).
    •   HasCrCard: Наличие кредитной карты у клиента (1 = да, 0 = нет).
    •   IsActiveMember: Является ли клиент активным членом банка (1 = да, 0 = нет).
    •   EstimatedSalary: Предполагаемая заработная плата клиента.
    •   Exited: Ушёл ли клиент (1 = да, 0 = нет), таргет.

Задача - бинарная классификация, предсказывание вероятности ухода клиента для тестового набора данных

В качестве baseline построен sample_submission.csv, в котором вероятность ухода каждого клиента - 0.5


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

In [ ]:
!pip install ydata-profiling --quiet

In [ ]:
!gdown 1XCFRgfoG0mK08v1tDCgJXBcPK6PbaVQ7 # test
!gdown 1ItKuMwuaqWcHCCmJ7Wj8neNgs1PPrpLy # train
!gdown 1lX2th7npV67Qzd1NLgv3rvEJyHuu4ZBD # test_submission

### Предобработка датасета

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
target = "Exited"

In [ ]:
def encode_categorical_features(train, test=None, categorical_features=['Geography', 'Gender']):
    """
    Кодирует категориальные признаки с помощью LabelEncoder
    """
    label_encoders = {}

    for feature in categorical_features:
        le = LabelEncoder()
        # Обработка train
        train[feature] = le.fit_transform(train[feature])

        # Обработка test
        if test is not None and feature in test.columns:
            test[feature] = le.transform(test[feature])

        label_encoders[feature] = le

    if test is not None:
        return train, test, label_encoders
    else:
        return train, label_encoders

In [ ]:
train, _ = encode_categorical_features(train)

In [ ]:
to_drop = ['id', 'CustomerId', 'Surname']
train = train.drop(columns=to_drop)

## EDA

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(train, title="Final_Project")

profile.to_notebook_iframe()
profile.to_file("eda_report.html")

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
num_cols = train.select_dtypes(include=['number']).columns.tolist()

In [ ]:
# График дисбаланса классов целевой переменной
target_cnt = train[target].value_counts()

plt.figure(figsize=(8, 6))
sns.barplot(x=target_cnt.index, y=target_cnt.values)
plt.title('Дисбаланс классов целевой переменной')
plt.xlabel('Exited')
plt.ylabel('Количество клиентов')
plt.xticks([0, 1], ['Остался', 'Ушел'])

for i, v in enumerate(target_cnt.values):
    plt.text(i, v + 50, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Распределение фичей
num_cols_no_target = [col for col in num_cols if col != 'Exited']

train[num_cols_no_target].hist(bins=25, figsize=(15,10))
plt.tight_layout()
plt.show()

In [ ]:
nan_counts = train.isna().sum()
nan_counts

In [ ]:
# Расчет корреляции с целевой переменной
target = 'Exited'
corr = train.corr(method='pearson')
corr_target = (
    corr[target]
    .drop(target)
    .sort_values(ascending=False)
)

plt.figure(figsize=(6, 8))
sns.barplot(
    x=corr_target.values,
    y=corr_target.index,
    orient="h"
)
plt.axvline(0, color="k", linewidth=0.8)
plt.title("Target correlation")
plt.tight_layout()
plt.show()

# Работа с аномалиями

In [ ]:
# Вспомогательные импорты
%matplotlib inline
from IPython.display import Image, display
from scipy.stats import zscore, t
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor, NearestNeighbors

def save_and_display(fig, path):
    fig.savefig(path, bbox_inches="tight", dpi=120)
    plt.close(fig)
    display(Image(path))

In [ ]:
# Классическая обработка выбросов
num_all = train.select_dtypes(include=["number"]).columns.tolist()
num_all = [c for c in num_all if c != target]
binary_cols = [c for c in num_all if train[c].nunique() <= 2]
numeric_for_outliers = [c for c in num_all if c not in binary_cols]

for df in (train, test):
    df[numeric_for_outliers] = df[numeric_for_outliers].apply(pd.to_numeric, errors="coerce")

train_mean = train[numeric_for_outliers].mean()
train_std = train[numeric_for_outliers].std(ddof=1)

z_mask_train = pd.DataFrame(
    np.abs((train[numeric_for_outliers] - train_mean) / train_std) > 3,
    columns=numeric_for_outliers,
)
z_mask_test = pd.DataFrame(
    np.abs((test[numeric_for_outliers] - train_mean) / train_std) > 3,
    columns=numeric_for_outliers,
)

q1 = train[numeric_for_outliers].quantile(0.25)
q3 = train[numeric_for_outliers].quantile(0.75)
iqr = q3 - q1
lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
iqr_mask_train = pd.DataFrame(
    (train[numeric_for_outliers] < lower) | (train[numeric_for_outliers] > upper),
    columns=numeric_for_outliers,
)
iqr_mask_test = pd.DataFrame(
    (test[numeric_for_outliers] < lower) | (test[numeric_for_outliers] > upper),
    columns=numeric_for_outliers,
)

def grubbs_threshold(series, alpha=0.05):
    x = series.dropna()
    if len(x) < 3:
        return np.inf
    n = len(x)
    t_crit = t.ppf(1 - alpha / (2 * n), n - 2)
    return ((n - 1) / np.sqrt(n)) * np.sqrt(t_crit**2 / (n - 2 + t_crit**2))

gcrit = train[numeric_for_outliers].apply(grubbs_threshold)
gstat_train = np.abs(train[numeric_for_outliers] - train_mean) / train_std
gstat_test = np.abs(test[numeric_for_outliers] - train_mean) / train_std
grubbs_mask_train = gstat_train.gt(gcrit)
grubbs_mask_test = gstat_test.gt(gcrit)

In [ ]:
# Сводка по выбросам
stat_outlier_mask_train = z_mask_train | iqr_mask_train | grubbs_mask_train
stat_outlier_mask_test = z_mask_test | iqr_mask_test | grubbs_mask_test

for df, mask, name in [
    (train, stat_outlier_mask_train, "train"),
    (test, stat_outlier_mask_test, "test"),
]:
    df["stat_outlier_any"] = mask.any(axis=1).astype(int)
    df["stat_outlier_count"] = mask.sum(axis=1)
    for col in numeric_for_outliers:
        df[f"{col}_outlier"] = mask[col].astype(int)

print("Исключены бинарные признаки:", binary_cols)
print("Доля строк с хотя бы одним выбросом (train):", train["stat_outlier_any"].mean().round(3))
print("Топ признаков по количеству выбросов (train):")
display(stat_outlier_mask_train.sum().sort_values(ascending=False).head())

In [ ]:
# Визуализация выбросов (взяты со сводки)
cols_to_plot = ["Age", "NumOfProducts", "CreditScore", "Tenure"]
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
for ax, col in zip(axes.ravel(), cols_to_plot):
    sns.boxplot(x=train[col], ax=ax, color="#7fb3d5")
    sns.stripplot(
        x=train[col],
        data=train[train[f"{col}_outlier"] == 1],
        ax=ax,
        color="red",
        size=3,
        alpha=0.6,
        label="Выбросы",
    )
    ax.set_title(f"{col}: выбросы по Z/IQR/Grubbs")
    ax.legend(loc="upper right")
plt.tight_layout()
save_and_display(fig, "outliers_boxplot.png")


**Age**: выбросы с краю (58+ лет). Их немного и, в целом, бизнес‑логика допускает пожилых клиентов. Вероятность шумовых записей низкая, поэтому оставим как есть, но используем флаг Age_outlier

**NumOfProducts**: выбросы на значении 4 (редкий пакет из 4 продуктов). Это край редкого сегмента, но бизнес‑возможный. Удалять не нужно: оставим и будем использовать флаг NumOfProducts_outlier как индикатор спецпредложения

**CreditScore**: выбросы в районе низких (<450) и высоких (>850) значений. Это разумные экстремумы (очень плохой / очень хороший скоринг). Удалять не стоит, так что тоже добавим флаг CreditScore_outlier

**Tenure**: выброс вообще всего один. Удалять не нужно, но на всякий случай отметим флагом

In [ ]:
# Признак плотности (kNN дистанции)
scaler = StandardScaler()
X_train = scaler.fit_transform(train[numeric_for_outliers])
X_test = scaler.transform(test[numeric_for_outliers].fillna(0))
nn = NearestNeighbors(n_neighbors=5).fit(X_train)
train_dists, _ = nn.kneighbors(X_train)
test_dists, _ = nn.kneighbors(X_test)
train["knn_mean_dist"] = train_dists[:, 1:].mean(axis=1)
test["knn_mean_dist"] = test_dists[:, 1:].mean(axis=1)

In [ ]:
# ML-методы выбросов (IsolationForest, LOF)
iso = IsolationForest(contamination=0.02, random_state=42)
train["iso_outlier"] = (iso.fit_predict(X_train) == -1).astype(int)
train["iso_score"] = iso.decision_function(X_train)
test["iso_outlier"] = (iso.predict(X_test) == -1).astype(int)
test["iso_score"] = iso.decision_function(X_test)

lof = LocalOutlierFactor(n_neighbors=20, contamination=0.02, novelty=True)
lof.fit(X_train)
train["lof_outlier"] = (lof.predict(X_train) == -1).astype(int)
train["lof_score"] = -lof.decision_function(X_train)
test["lof_outlier"] = (lof.predict(X_test) == -1).astype(int)
test["lof_score"] = -lof.decision_function(X_test)

In [ ]:
# Визуализация ML-меток
fig, ax = plt.subplots(figsize=(10, 4))
sns.scatterplot(
    x=train["CreditScore"],
    y=train["Balance"],
    hue=train["iso_outlier"],
    palette=["steelblue", "red"],
    alpha=0.6,
    ax=ax,
)
ax.set_title("Isolation Forest: CreditScore vs Balance")
save_and_display(fig, "iso_cs_balance.png")

fig, ax = plt.subplots(figsize=(10, 4))
sns.scatterplot(
    x=train["Age"],
    y=train["EstimatedSalary"],
    hue=train["lof_outlier"],
    palette=["steelblue", "darkorange"],
    alpha=0.6,
    ax=ax,
)
ax.set_title("Local Outlier Factor: Age vs EstimatedSalary")
save_and_display(fig, "lof_age_salary.png")


In [ ]:
# Рассчитываем метрики, считая что целевые аномалии — это реальные уходы клиентов
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

y_true = train[target].astype(int)

def print_metrics(name, y_pred, score):
    p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)
    auc = roc_auc_score(y_true, score)
    print(f"{name}: precision={p:.3f}, recall={r:.3f}, f1={f1:.3f}, roc_auc={auc:.3f}")

print_metrics("Isolation Forest", train["iso_outlier"], -train["iso_score"])  # чем ниже score, тем аномальнее
print_metrics("LOF", train["lof_outlier"], -train["lof_score"]) # выше lof_score = более аномально

both_anom = (train["iso_outlier"] == 1) & (train["lof_outlier"] == 1)
only_iso = (train["iso_outlier"] == 1) & (train["lof_outlier"] == 0)
only_lof = (train["iso_outlier"] == 0) & (train["lof_outlier"] == 1)

print("\nДоля пересечения (оба метят):", both_anom.mean().round(3))
print("Доля только IsolationForest:", only_iso.mean().round(3))
print("Доля только LOF:", only_lof.mean().round(3))

print("\nПересечение с таргетом среди обоих методов:")
display(pd.crosstab(both_anom, y_true, normalize="index"))

print("Пересечение с таргетом среди only_iso:")
display(pd.crosstab(only_iso, y_true, normalize="index"))

print("Пересечение с таргетом среди only_lof:")
display(pd.crosstab(only_lof, y_true, normalize="index"))


Isolation Forest ловит больше целевых: LOF существенно слабее. Для признаков лучше использовать метки IF, а пересечение IF и LOF можно трактовать как жёсткие аномалии с высоким churn. LOF сейчас отдельно несёт мало пользы при текущих параметрах

## 2.2 Генерация признаков и отбор переменных


In [ ]:
#Шаг 1. Обработайте категориальные переменные
#Тarget Encoding для одной переменной

from sklearn.model_selection import StratifiedKFold
import numpy as np

target = 'Exited'
te_col = 'Geography'
te_new_col = te_col + '_te'

train[te_new_col] = np.nan
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, val_idx in kf.split(train, train[target]):
    fold_train = train.iloc[train_idx]
    fold_mapping = fold_train.groupby(te_col)[target].mean()

    train.loc[val_idx, te_new_col] = train.loc[val_idx, te_col].map(fold_mapping)

global_mapping = train.groupby(te_col)[target].mean()
global_mean = train[target].mean()

test[te_new_col] = test[te_col].map(global_mapping)

train[te_new_col].fillna(global_mean, inplace=True)
test[te_new_col].fillna(global_mean, inplace=True)

In [ ]:
#Лодировка остальных категориальных признаков

from sklearn.preprocessing import LabelEncoder

categorical_cols = ['Gender']
label_encoders = {}

Geography - target encoding, потому что мало категорий, но связь с таргетом может быть сложной, TE даёт компактный числовой признак, хорошо работающий и с линейными моделями, и нет


In [ ]:
#Зафиксировать промежуточные метрики

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

feature_cols = [c for c in train.columns if c != target]

X = train[feature_cols]
y = train[target]

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

log_reg = LogisticRegression(
    max_iter=2000,
    class_weight='balanced',
    n_jobs=-1,
    random_state=42
)

log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_valid)
y_proba = log_reg.predict_proba(X_valid)[:, 1]

roc_auc = roc_auc_score(y_valid, y_proba)
print('roc_auc:', roc_auc)
print()
print(classification_report(y_valid, y_pred))

После кодировки категориальных признаков и обучения базовой логистической регрессии с учётом дисбаланса классовмодель показала:

ROC-AUC ≈ 0.89

accuracy ≈ 0.81

recall по классу "ушел" ≈ 0.81, precision ≈ 0.52.

Это означает, что модель хорошо различает склонных к уходу клиентов и позволяет находить около 80% реально уходящих клиентов, хотя при этом часть оставшихся помечается как потенциально уходящие. Эти значения будут использоваться как промежуточная точка сравнения при дальнейшем обогащении признаков и их отборе.

In [ ]:
#Шаг 2. Добавьте признаки, основанные на ближайших соседях

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

knn_features = numeric_for_outliers

medians = train[knn_features].median()

train_knn = train[knn_features].fillna(medians)
test_knn = test[knn_features].fillna(medians)

scaler_knn = StandardScaler()
X_train_knn = scaler_knn.fit_transform(train_knn)
X_test_knn = scaler_knn.transform(test_knn)

k = 5
nn = NearestNeighbors(n_neighbors=k)
nn.fit(X_train_knn)

train_dists, _ = nn.kneighbors(X_train_knn)
test_dists, _ = nn.kneighbors(X_test_knn)

train_dists_no_self = train_dists[:, 1:]

train['knn_mean_dist'] = train_dists_no_self.mean(axis=1)
test['knn_mean_dist'] = test_dists.mean(axis=1)

train['knn_min_dist'] = train_dists_no_self.min(axis=1)
test['knn_min_dist'] = test_dists.min(axis=1)

train['knn_max_dist'] = train_dists_no_self.max(axis=1)
test['knn_max_dist'] = test_dists.max(axis=1)


Перед построением признаков на основе ближайших соседей были заполнены пропуски в числовых переменных. Для каждого признака использовалась медиана по обучающей выборке, которая затем применялась как к train, так и к test. Это позволило корректно использовать методы, не поддерживающие значения NaN (например, NearestNeighbors), и при этом не использовать информацию из тестовой выборки.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report

target = 'Exited'
feature_cols = [c for c in train.columns if c != target]

X = train[feature_cols]
y = train[target]

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

log_reg = LogisticRegression(
    max_iter=2000,
    class_weight='balanced',
    n_jobs=-1,
    random_state=42
)

log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_valid)
y_proba = log_reg.predict_proba(X_valid)[:, 1]

roc_auc = roc_auc_score(y_valid, y_proba)
print('roc_auc:', roc_auc)
print()
print(classification_report(y_valid, y_pred))


Шаг 3. Обработка временных признаков

В исходном датасете отсутствуют явные временные признаки.
Переменная `Tenure` отражает стаж клиента в банке в годах и уже используется как обычный числовой признак, но не является периодической временной компонентой (её нет смысла разворачивать через синус/косинус-кодирование).



In [ ]:
#Шаг 4. Сформируйте контекстные признаки, отражающие специфику вашей задачи

train['balance_to_salary'] = train['Balance'] / (train['EstimatedSalary'] + 1)
test['balance_to_salary'] = test['Balance'] / (test['EstimatedSalary'] + 1)

high_balance_threshold = train['Balance'].quantile(0.9)

train['is_high_value_client'] = (
    (train['Balance'] > high_balance_threshold) & (train['NumOfProducts'] > 1)
).astype(int)

test['is_high_value_client'] = (
    (test['Balance'] > high_balance_threshold) & (test['NumOfProducts'] > 1)
).astype(int)

train['is_new_client'] = (train['Tenure'] <= 1).astype(int)
test['is_new_client'] = (test['Tenure'] <= 1).astype(int)

train['is_senior'] = (train['Age'] >= 60).astype(int)
test['is_senior'] = (test['Age'] >= 60).astype(int)

balance_to_salary - это отношение баланса клиента к его оценочной зарплате.
Мы предполагаем, что клиенты, у которых на счетах лежит аномально большой объем средств относительно их дохода, могут быть более ценными и более чувствительными к условиям банка, и это может влиять на вероятность ухода

is_high_value_client - бинарный признак, равный 1 - если клиент входит в верхние 10% по балансу и одновременно пользуется более чем одним продуктом банка.
Мы предполагаем, что премиальные клиенты ведут себя иначе, чем обычные, например, их отток особенно важен для банка, и их поведение может отличаться от остальной массы

is_new_client - индикатор новых клиентов (стаж в банке <= 1 год).
Мы предполагаем, что клиенты в начале жизненного цикла чаще уходят, поэтому ранний отток рассматриваем как отдельный сценарий

is_senior - индикатор клиентов старшего возраста (Age >= 60).
Мы предполагаем, что поведение и потребности возрастных клиентов могут отличаться от более молодых, что может отражаться на вероятности ухода


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report

target = 'Exited'
feature_cols = [c for c in train.columns if c != target]

X = train[feature_cols]
y = train[target]

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

log_reg = LogisticRegression(
    max_iter=2000,
    class_weight='balanced',
    n_jobs=-1,
    random_state=42
)

log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_valid)
y_proba = log_reg.predict_proba(X_valid)[:, 1]

roc_auc = roc_auc_score(y_valid, y_proba)
print('roc_auc:', roc_auc)
print()
print(classification_report(y_valid, y_pred))


После добавления признаков на основе kNN и контекстных бизнес-гипотез качество базовой логистической регрессии практически не изменилось: ROC-AUC вырос с ≈0.8922 до ≈0.8925, а остальные метрики остались на похожем уровне.

Это значит, что исходный набор признаков уже содержал большую часть информации, важной для линейной модели. Однако новые признаки могут оказаться более полезными для нелинейных моделей, а также для интерпретации поведения разных сегментов клиентов.

In [ ]:
target = 'Exited'
feature_cols = [c for c in train.columns if c != target]

X = train[feature_cols].copy()
y = train[target].copy()

X = X.fillna(X.median())

In [ ]:
#Отбор через фильтры

import pandas as pd
import numpy as np

numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

corr_with_target = X[numeric_cols].corrwith(y).abs().sort_values(ascending=False)
corr_with_target.head(20)


In [ ]:
#χ²

from sklearn.feature_selection import chi2, SelectKBest
from sklearn.preprocessing import MinMaxScaler

scaler_mm = MinMaxScaler()
X_chi2 = scaler_mm.fit_transform(X)

chi2_selector = SelectKBest(score_func=chi2, k='all')
chi2_selector.fit(X_chi2, y)

chi2_scores = pd.Series(chi2_selector.scores_, index=feature_cols).sort_values(ascending=False)
chi2_scores.head(20)


In [ ]:
#ANOVA

from sklearn.feature_selection import f_classif

f_selector = SelectKBest(score_func=f_classif, k='all')
f_selector.fit(X, y)

anova_scores = pd.Series(f_selector.scores_, index=feature_cols).sort_values(ascending=False)
anova_scores.head(20)

Все три фильтра говорят одно и то же:

Очень важные признаки: Age, NumOfProducts, Geography_te, IsActiveMember, iso_score и iso_outlier, knn (mean/max/min dist), бинарки аномалий (stat_outlier_any, stat_outlier_count, Age_outlier, lof_outlier), Gender, Balance

Менее важные: сырой Geography, CreditScore, более слабые флаги вроде is_new_client, lof_score

In [ ]:
#Обертки: RFECV с логистической регрессией

from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

base_clf = LogisticRegression(
    max_iter=2000,
    class_weight='balanced',
    n_jobs=-1,
    random_state=42
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rfecv = RFECV(
    estimator=base_clf,
    step=1,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1
)

rfecv.fit(X, y)

selected_rfecv = [feature_cols[i] for i, flag in enumerate(rfecv.support_) if flag]

print('Оптимальное число признаков по RFECV:', rfecv.n_features_)
print('Признаки, отобранные RFECV:')
selected_rfecv


In [ ]:
#L1-регуляризация

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

l1_clf = LogisticRegression(
    penalty='l1',
    solver='liblinear',
    class_weight='balanced',
    max_iter=2000,
    random_state=42
)

l1_clf.fit(X, y)

coef_abs = np.abs(l1_clf.coef_[0])
l1_importance = pd.Series(coef_abs, index=feature_cols).sort_values(ascending=False)

selected_l1 = l1_importance[l1_importance > 0].index.tolist()

print('Количество признаков с ненулевым весом (L1):', len(selected_l1))
print('Топ по важности (L1):')
l1_importance.head(20)


In [ ]:
#Feature importances Random Forest

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=7,
    min_samples_leaf=20,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf.fit(X, y)

rf_importances = pd.Series(rf.feature_importances_, index=feature_cols).sort_values(ascending=False)

print('Топ-20 признаков по важности:')
rf_importances.head(20)


In [ ]:
#Сбор финального списка признаков
top_rf_features = rf_importances.head(20).index.tolist()

selected_features = sorted(set(selected_rfecv) | set(selected_l1) | set(top_rf_features))

print('Итого признаков:', len(selected_features))
selected_features

In [ ]:
#Нестабильные во времени признаки

from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd

kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
importances_per_fold = []

for train_idx, val_idx in kf.split(X, y):
    X_tr, y_tr = X.iloc[train_idx], y.iloc[train_idx]

    rf_fold = RandomForestClassifier(
        n_estimators=300,
        max_depth=7,
        min_samples_leaf=20,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    )
    rf_fold.fit(X_tr, y_tr)
    importances_per_fold.append(rf_fold.feature_importances_)

importances_per_fold = np.vstack(importances_per_fold)

importances_mean = importances_per_fold.mean(axis=0)
importances_std = importances_per_fold.std(axis=0)

instability = importances_std / (importances_mean + 1e-6)
instability_series = pd.Series(instability, index=feature_cols).sort_values(ascending=False)

print('Топ-20 наиболее нестабильных признаков):')
instability_series.head(20)

In [ ]:
selected_features = [
    'Geography',
    'Gender',
    'Age',
    'Tenure',
    'NumOfProducts',
    'HasCrCard',
    'IsActiveMember',
    'stat_outlier_any',
    'stat_outlier_count',
    'CreditScore_outlier',
    'Age_outlier',
    'Tenure_outlier',
    'NumOfProducts_outlier',
    'knn_mean_dist',
    'iso_outlier',
    'iso_score',
    'lof_outlier',
    'lof_score',
    'Geography_te',
    'knn_min_dist',
    'knn_max_dist',
    'balance_to_salary',
    'is_high_value_client',
    'is_new_client',
    'is_senior'
]

Фильтровые подходы показали, что сильнее всего с таргетом связаны базовые признаки (`Age`, `NumOfProducts`, `IsActiveMember`, `Balance`, `Gender`) и сгенерированные фичи. При этом исходные `Geography` и `CreditScore` оказались менее информативными

RFECV на логистической регрессии отобрал 25 признаков, включающих основные параметры клиента, географию после TE, признаки аномальности, kNN-фичи и контекстные переменные вроде `balance_to_salary` и индикаторов разных групп клиентов.

Встроенные методы (L1 и RandomForest) в целом подтвердили важность тех же признаков. Некоторые редкие бинарные признаки оказались менее стабильными.

Анализ нестабильности через разные фолды показал, что сильнее всего меняются редкие флаги и аномальные признаки, тогда как ключевые фичи остаются устойчивыми.

В итоге для дальнейшего моделирования выбран набор признаков, который подтверждён всеми тремя подходами и хорошо отражает как основные характеристики клиента, так и важные гипотезы о его поведении.


# Интерпретация и диагностика моделей

## Шаг 1. Проинтерпретируйте модели

In [ ]:
#Подготовка данных для интерпретаций SHAP и LIME
target = 'Exited'
feature_cols = [c for c in train.columns if c != target]

X_full = train[feature_cols].copy()
y_full = train[target].copy()

X_full = X_full.fillna(X_full.median(numeric_only=True))

X_train, X_valid, y_train, y_valid = train_test_split(
    X_full,
    y_full,
    test_size=0.2,
    stratify=y_full,
    random_state=42
)

X_train.shape, X_valid.shape

In [ ]:
#обучаем модели на финальном наборе признаков,
#чтобы интерпретация была огласованной

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

log_reg = LogisticRegression(
    max_iter=2000,
    class_weight='balanced',
    n_jobs=-1,
    random_state=42
)

log_reg.fit(X_train, y_train)

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=7,
    min_samples_leaf=20,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

In [ ]:
#shap для логистической регрессии
import shap

shap.initjs()
explainer_log_reg = shap.LinearExplainer(
    log_reg,
    X_train,
    feature_perturbation='interventional'
)

shap_values_log_reg = explainer_log_reg.shap_values(X_valid)

shap.summary_plot(
    shap_values_log_reg,
    X_valid,
    plot_type='bar',
    show=True
)
shap.summary_plot(
    shap_values_log_reg,
    X_valid,
    show=True
)

Тут все довольно ожидаемо. Самые важные признаки: Age, NumOfProducts и IsActiveMember. По второму графику видно, что большой возраст увеличивает вероятность ухода, а большое число продуктов и активность клиента наоборот уменьшают вероятность ухода.
Признаки вроде Gender, CreditScore и kNN-фичи тоже что-то дают модели, но заметно слабее. Остальные признаки почти не влияют.

SHAP показывает, что модель живет довольно простой логикой

In [ ]:
#shap для RandomFores
import shap

explainer_rf = shap.TreeExplainer(rf)
shap_values_rf = explainer_rf.shap_values(X_valid)

if isinstance(shap_values_rf, list):
    shap_values_rf_class1 = shap_values_rf[1]
else:
    if shap_values_rf.ndim == 3:
        shap_values_rf_class1 = shap_values_rf[:, :, 1]
    else:
        shap_values_rf_class1 = shap_values_rf

#графики
shap.summary_plot(
    shap_values_rf_class1,
    X_valid,
    plot_type='bar',
    show=True
)

shap.summary_plot(
    shap_values_rf_class1,
    X_valid,
    show=True
)

RandomForest подтверждает то же, что и линейная модель - главное: возраст, продукты и активность

In [ ]:
!pip -q install lime

In [ ]:
#настройка LIME

import lime
from lime.lime_tabular import LimeTabularExplainer

class_names = ['stayed', 'exited']
lime_explainer = LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns.tolist(),
    class_names=class_names,
    mode='classification',
    discretize_continuous=True
)

In [ ]:
#lime для логистической регрессии

n_samples = min(200, X_valid.shape[0])
np.random.seed(42)

sample_indices = np.random.choice(
    X_valid.index,
    size=n_samples,
    replace=False
)

lime_importances_log = pd.Series(
    0.0,
    index=X_train.columns
)

for idx in sample_indices:
    exp = lime_explainer.explain_instance(
        data_row=X_valid.loc[idx].values,
        predict_fn=log_reg.predict_proba,
        num_features=10
    )
    for feat_idx, weight in exp.local_exp[1]:
        feat_name = X_train.columns[feat_idx]
        lime_importances_log[feat_name] += abs(weight)

lime_importances_log = lime_importances_log / n_samples
lime_importances_log = lime_importances_log.sort_values(ascending=False)

top_n = 20
plt.figure(figsize=(8, 6))
lime_importances_log.head(top_n).plot(kind='barh')
plt.gca().invert_yaxis()
plt.title('LIME: глобальная важность признаков для логистической регрессии')
plt.xlabel('среднее |влияние| по объектам')
plt.tight_layout()
plt.show()

Снова ничего нового. По графику видно, что модель сильнее всего опирается на NumOfProducts, а также на IsActiveMember, Age, Gender и balance_to_salary.
Фичи, связанные с выбросами и аномалиями, появляются в списке, но их вклад скорее вторичный.

In [ ]:
#lime для RandomForest

lime_importances_rf = pd.Series(
    0.0,
    index=X_train.columns
)

for idx in sample_indices:
    exp = lime_explainer.explain_instance(
        data_row=X_valid.loc[idx].values,
        predict_fn=rf.predict_proba,
        num_features=10
    )
    for feat_idx, weight in exp.local_exp[1]:
        feat_name = X_train.columns[feat_idx]
        lime_importances_rf[feat_name] += abs(weight)

lime_importances_rf = lime_importances_rf / n_samples
lime_importances_rf = lime_importances_rf.sort_values(ascending=False)

top_n = 20
plt.figure(figsize=(8, 6))
lime_importances_rf.head(top_n).plot(kind='barh')
plt.gca().invert_yaxis()
plt.title('LIME: глобальная важность признаков для RandomForest')
plt.xlabel('среднее |влияние| по объектам')
plt.tight_layout()
plt.show()

В целом LIME для RandomForest подтверждает ту же общую логику, что и все остальное

**Общий вывод:**

По результатам SHAP и LIME видно, что ключевые признаки у линейной и ансамблевой моделей в целом совпадают. В обоих случаях в топе стабильно находятся Age, NumOfProducts и IsActiveMember

Направления влияния тоже согласуются: большой возраст увеличивает вероятность ухода, большое число продуктов снижает вероятность ухода, активность клиента увеличивает вероятность ухода.

Второй уровень признаков (например, Gender, частично CreditScore, некоторые kNN-фичи) тоже встречается везде, но их вклад слабее. Признаки аномалий и бинарные флаги, в целом, показывают минимальное влияние.


In [ ]:
#выбор одного наблюдения
#выберем клиента, у которого вероятность ухода высокая для лучшей интерпретации

proba_rf = rf.predict_proba(X_valid)[:, 1]
idx = X_valid.index[np.argmax(proba_rf)]

x_one = X_valid.loc[idx]
print('chosen index:', idx)
print('rf proba exited:', rf.predict_proba(x_one.to_frame().T)[0, 1])
print('logreg proba exited:', log_reg.predict_proba(x_one.to_frame().T)[0, 1])

In [ ]:
def logreg_predict_proba_with_names(x):
    x_df = pd.DataFrame(x, columns=X_train.columns)
    return log_reg.predict_proba(x_df)

def rf_predict_proba_with_names(x):
    x_df = pd.DataFrame(x, columns=X_train.columns)
    return rf.predict_proba(x_df)

In [ ]:
#lime локальное объяснение

exp_log = lime_explainer.explain_instance(
    data_row=x_one.values,
    predict_fn=logreg_predict_proba_with_names,
    num_features=10
)
exp_rf = lime_explainer.explain_instance(
    data_row=x_one.values,
    predict_fn=rf_predict_proba_with_names,
    num_features=10
)

print('LIME explanation for LogisticRegression:')
print(exp_log.as_list())

print('\nLIME explanation for RandomForest:')
print(exp_rf.as_list())

Обе модели объясняют высокий риск ухода почти одинаково. Главные факторы не поменялись (это NumOfProducts ≤ 1, Age > 42, IsActiveMember = 0 и Gender) Эти признаки сильнее всего сдвигают предсказание в сторону класса "ушел".

В RandomForest аналогично


In [ ]:
#SHAP LogisticRegression локальное объяснение

explainer_log_reg = shap.LinearExplainer(
    log_reg,
    X_train,
    feature_perturbation='interventional'
)

shap_values_log = explainer_log_reg.shap_values(x_one.to_frame().T)

shap.plots.waterfall(
    shap.Explanation(
        values=shap_values_log[0],
        base_values=explainer_log_reg.expected_value,
        data=x_one.values,
        feature_names=X_train.columns
    )
)

Главный признак: Age = 60. Также заметно повышают риск NumOfProducts = 1, IsActiveMember = 0, Gender = 0 и относительно низкий CreditScore около 598.

При этом признаки, связанные с выбросами (Age_outlier, stat_outlier_any/count, is_senior)  немного тормозят модель, но не перекрывают сильное влияние других признаков.


In [ ]:
#SHAP RandomForest локальное объяснение

explainer_rf = shap.TreeExplainer(rf)
sv = explainer_rf.shap_values(x_one.to_frame().T)

if isinstance(sv, list):
    sv_class1 = sv[1][0]
    base_val = explainer_rf.expected_value[1]
else:
    if sv.ndim == 3:
        sv_class1 = sv[0, :, 1]
        base_val = explainer_rf.expected_value[1]
    else:
        sv_class1 = sv[0]
        base_val = explainer_rf.expected_value

shap.plots.waterfall(
    shap.Explanation(
        values=sv_class1,
        base_values=base_val,
        data=x_one.values,
        feature_names=X_train.columns
    )
)

То же самое, только вклад распределен распределен ровнее, сильнее видно Geography и немного kNN

**Общий вывод**

Для клиента LIME и SHAP дают похожую картину для логистической регрессии и для RandomForest. В обоих методах ключевыми факторами стали большой возраст, малое число продуктов и неактивность клиента, то есть основные причины высокого риска ухода совпадают.


## Построение SHAP-эмбеддингов и анализ сдвигов

In [ ]:
# 1. Получение SHAP-эмбеддингов
def get_shap_embeddings(model, X, explainer_type='tree'):
    if explainer_type == 'tree':
        explainer = shap.TreeExplainer(model)
    elif explainer_type == 'linear':
        explainer = shap.LinearExplainer(model, X, feature_perturbation='interventional')
    else:
        raise ValueError("Unsupported explainer type")

    shap_values = explainer.shap_values(X)

    if isinstance(shap_values, list):
        shap_embeddings = shap_values[1]
    elif shap_values.ndim == 3:
        shap_embeddings = shap_values[:, :, 1]
    else:
        shap_embeddings = shap_values

    return shap_embeddings, explainer.expected_value

shap_train_rf, base_value_rf = get_shap_embeddings(rf, X_train)
shap_valid_rf, _ = get_shap_embeddings(rf, X_valid)

shap_train_df = pd.DataFrame(shap_train_rf, columns=X_train.columns)
shap_valid_df = pd.DataFrame(shap_valid_rf, columns=X_valid.columns)

In [ ]:
# 2. Выявление сдвигов и аномалий в SHAP-пространстве
from scipy import stats

drift_results = {}
for col in shap_train_df.columns:
    stat, p_value = stats.ks_2samp(shap_train_df[col], shap_valid_df[col])
    drift_results[col] = {'statistic': stat, 'p_value': p_value}

drift_df = pd.DataFrame(drift_results).T
print("Признаки с наибольшим сдвигом в SHAP-пространстве:")
print(drift_df.nlargest(10, 'statistic'))

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
top_drift_features = drift_df.nlargest(6, 'statistic').index

for idx, feature in enumerate(top_drift_features):
    ax = axes[idx // 3, idx % 3]
    ax.hist(shap_train_df[feature], alpha=0.5, label='Train', bins=30, density=True)
    ax.hist(shap_valid_df[feature], alpha=0.5, label='Valid', bins=30, density=True)
    ax.set_title(f'{feature}\nKS stat: {drift_df.loc[feature, "statistic"]:.3f}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 3. Очистка данных на основе анализа сдвигов
threshold = 3
outlier_mask = np.zeros(len(shap_train_df), dtype=bool)

for col in shap_train_df.columns:
    col_mean = shap_train_df[col].mean()
    col_std = shap_train_df[col].std()
    col_outliers = np.abs(shap_train_df[col] - col_mean) > threshold * col_std
    outlier_mask = outlier_mask | col_outliers

print(f"Доля выбросов в SHAP-пространстве: {outlier_mask.mean():.3%}")

outlier_mask_series = pd.Series(outlier_mask, index=X_train.index).astype(bool)

X_train_clean = X_train[~outlier_mask_series].copy()
y_train_clean = y_train[~outlier_mask_series].copy()

In [ ]:
# 4. Переобучение модели на очищенных данных
rf_clean = RandomForestClassifier(
    n_estimators=300,
    max_depth=7,
    min_samples_leaf=20,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_clean.fit(X_train_clean, y_train_clean)

In [ ]:
# Сравнение метрик до и после очистки
from sklearn.metrics import roc_auc_score, f1_score

y_pred_original = rf.predict(X_valid)
y_pred_clean = rf.predict(X_valid)

y_proba_original = rf.predict_proba(X_valid)[:, 1]
y_proba_clean = rf.predict_proba(X_valid)[:, 1]

roc_auc_original = roc_auc_score(y_valid, y_proba_original)
roc_auc_clean = roc_auc_score(y_valid, y_proba_clean)

f1_original = f1_score(y_valid, y_pred_original)
f1_clean = f1_score(y_valid, y_pred_clean)

print("Сравнение метрик до и после очистки:")
print(f"ROC-AUC: {roc_auc_original:.4f} → {roc_auc_clean:.4f} (Δ{roc_auc_clean - roc_auc_original:+.4f})")
print(f"F1-score: {f1_original:.4f} → {f1_clean:.4f} (Δ{f1_clean - f1_original:+.4f})")

In [ ]:
# 5. Кластеризация SHAP-эмбеддингов
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap.umap_ as umap

pca = PCA(n_components=2, random_state=42)
shap_pca = pca.fit_transform(shap_train_df)

umap_reducer = umap.UMAP(n_components=2, random_state=42)
shap_umap = umap_reducer.fit_transform(shap_train_df)

kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(shap_train_df)

dbscan = DBSCAN(eps=0.5, min_samples=10)
dbscan_labels = dbscan.fit_predict(shap_train_df)

agglo = AgglomerativeClustering(n_clusters=4)
agglo_labels = agglo.fit_predict(shap_train_df)

In [ ]:
# Визуализация кластеров
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

scatter = axes[0, 0].scatter(shap_pca[:, 0], shap_pca[:, 1], c=kmeans_labels, cmap='tab10', alpha=0.6)
axes[0, 0].set_title('KMeans кластеризация (PCA)')
axes[0, 0].set_xlabel('PC1')
axes[0, 0].set_ylabel('PC2')
fig.colorbar(scatter, ax=axes[0, 0])

scatter = axes[0, 1].scatter(shap_umap[:, 0], shap_umap[:, 1], c=kmeans_labels, cmap='tab10', alpha=0.6)
axes[0, 1].set_title('KMeans кластеризация (UMAP)')
axes[0, 1].set_xlabel('UMAP1')
axes[0, 1].set_ylabel('UMAP2')
fig.colorbar(scatter, ax=axes[0, 1])

scatter = axes[1, 0].scatter(shap_pca[:, 0], shap_pca[:, 1], c=dbscan_labels, cmap='tab10', alpha=0.6)
axes[1, 0].set_title('DBSCAN кластеризация (PCA)')
axes[1, 0].set_xlabel('PC1')
axes[1, 0].set_ylabel('PC2')
fig.colorbar(scatter, ax=axes[1, 0])

scatter = axes[1, 1].scatter(shap_pca[:, 0], shap_pca[:, 1], c=agglo_labels, cmap='tab10', alpha=0.6)
axes[1, 1].set_title('Agglomerative кластеризация (PCA)')
axes[1, 1].set_xlabel('PC1')
axes[1, 1].set_ylabel('PC2')
fig.colorbar(scatter, ax=axes[1, 1])

cluster_target_dist = []
for cluster_id in np.unique(kmeans_labels):
    mask = kmeans_labels == cluster_id
    churn_rate = y_train[mask].mean()
    cluster_target_dist.append((cluster_id, churn_rate, mask.sum()))

cluster_target_df = pd.DataFrame(cluster_target_dist,
                                 columns=['Cluster', 'Churn Rate', 'Size'])

axes[0, 2].barh(cluster_target_df['Cluster'].astype(str),
                cluster_target_df['Churn Rate'])
axes[0, 2].set_title('Churn Rate по кластерам (KMeans)')
axes[0, 2].set_xlabel('Churn Rate')

axes[1, 2].pie(cluster_target_df['Size'],
               labels=cluster_target_df['Cluster'].astype(str),
               autopct='%1.1f%%')
axes[1, 2].set_title('Размеры кластеров (KMeans)')

plt.tight_layout()
plt.show()

In [ ]:
# 6. Интерпретация кластеров
train_with_clusters = train.copy()
train_with_clusters['shap_cluster'] = -1
train_with_clusters.loc[X_train.index, 'shap_cluster'] = kmeans_labels

cluster_profiles = []
for cluster_id in np.unique(kmeans_labels):
    if cluster_id == -1:
        continue

    cluster_mask = train_with_clusters['shap_cluster'] == cluster_id
    cluster_data = train_with_clusters[cluster_mask]

    profile = {
        'cluster': cluster_id,
        'size': len(cluster_data),
        'churn_rate': cluster_data[target].mean()
    }

    key_features = ['Age', 'NumOfProducts', 'IsActiveMember', 'Balance', 'CreditScore']
    for feat in key_features:
        profile[f'{feat}_mean'] = cluster_data[feat].mean()

    cluster_profiles.append(profile)

cluster_profile_df = pd.DataFrame(cluster_profiles)
print("\nПрофили кластеров:")
print(cluster_profile_df.to_string())

In [ ]:
# 7. Добавление кластера как признака и переобучение
X_train_with_cluster = X_train.copy()
X_valid_with_cluster = X_valid.copy()

X_train_with_cluster['shap_cluster'] = kmeans_labels
X_valid_with_cluster['shap_cluster'] = kmeans.predict(shap_valid_df)

rf_with_cluster = RandomForestClassifier(
    n_estimators=300,
    max_depth=7,
    min_samples_leaf=20,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_with_cluster.fit(X_train_with_cluster, y_train)

y_proba_with_cluster = rf_with_cluster.predict_proba(X_valid_with_cluster)[:, 1]
roc_auc_with_cluster = roc_auc_score(y_valid, y_proba_with_cluster)

print(f"\nROC-AUC с кластерным признаком: {roc_auc_with_cluster:.4f}")
print(f"Изменение относительно исходной: {roc_auc_with_cluster - roc_auc_original:+.4f}")


Выводы:

    1. Очистка выбросов не изменила значение ROC-AUC
    2. Кластеризация SHAP-эмбеддингов выявила 4 кластера с разной долей оттока клиентов

## Валидация и применение SHAP-эмбеддингов

In [ ]:
# 1. Кросс-валидация с SHAP-эмбеддингами
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

X_original = X_train
X_shap_only = shap_train_df
X_combined = pd.concat([X_original.reset_index(drop=True),
                       shap_train_df.reset_index(drop=True)], axis=1)
X_combined.columns = [f"orig_{col}" for col in X_original.columns] + \
                     [f"shap_{col}" for col in shap_train_df.columns]

rf_cv = RandomForestClassifier(
    n_estimators=300,
    max_depth=7,
    min_samples_leaf=20,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

cv_scores_original = cross_val_score(rf_cv, X_original, y_train,
                                     cv=5, scoring='roc_auc', n_jobs=-1)
cv_scores_shap = cross_val_score(rf_cv, X_shap_only, y_train,
                                 cv=5, scoring='roc_auc', n_jobs=-1)
cv_scores_combined = cross_val_score(rf_cv, X_combined, y_train,
                                     cv=5, scoring='roc_auc', n_jobs=-1)

print("\nКросс-валидация ROC-AUC:")
print(f"Только исходные признаки: {cv_scores_original.mean():.4f} (±{cv_scores_original.std():.4f})")
print(f"Только SHAP-эмбеддинги: {cv_scores_shap.mean():.4f} (±{cv_scores_shap.std():.4f})")
print(f"Комбинация: {cv_scores_combined.mean():.4f} (±{cv_scores_combined.std():.4f})")

In [ ]:
# 2. Построение графа взаимосвязей признаков
import networkx as nx
from scipy.stats import spearmanr

corr_threshold = 0.3
G = nx.Graph()

for feature in X_train.columns:
    G.add_node(feature, type='feature')

for i, feat1 in enumerate(X_train.columns):
    for feat2 in X_train.columns[i+1:]:
        mask = X_train[feat1].notna() & X_train[feat2].notna()
        if mask.sum() > 10:
            corr, _ = spearmanr(X_train.loc[mask, feat1], X_train.loc[mask, feat2])
            if not np.isnan(corr) and abs(corr) > corr_threshold:
                G.add_edge(feat1, feat2, weight=abs(corr), correlation=corr)

In [ ]:
plt.figure(figsize=(14, 12))
pos = nx.spring_layout(G, seed=42, k=0.5, iterations=50)

nx.draw_networkx_nodes(
    G, pos,
    node_color='lightblue',
    node_size=800,
    alpha=0.8
)

edges = G.edges()
weights = [G[u][v]['weight'] * 5 for u, v in edges]
nx.draw_networkx_edges(
    G, pos,
    width=weights,
    alpha=0.5,
    edge_color='gray'
)

nx.draw_networkx_labels(
    G, pos,
    font_size=10,
    font_weight='bold'
)

plt.title(f'Граф взаимосвязей признаков (корреляция > {corr_threshold})', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

Выводы:

    1. Обучение на SHAP-эмбеддингах дало наилучший результат, ROC-AUC = 0.9481; на втором же месте - комбинация исходных данных и SHAP-эмбеддингов, ROC-AUC = 0.9463
    2. Был построен граф взаимосвязей признаков
